## A* Search

In [1]:
class Graph:
    
    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list
        
    def get_neighbors(self, v):
        return self.adjacency_list[v]
    
    # heuristic function with equal values for all nodes
    def h(self, n):
        H = {
            'A':1,
            'B':1,
            'C':1,
            'D':1,
        }
        return H[n]
    
    def a_star_algorithm(self, start_node, stop_node):
        # open_list is a list of node which have been visited, but who's neighbors
        # haven't all been inspected, starts off with the start node
        # closed_list is a list of nodes which have been visited
        # and who's neighbors have been inspected
        open_list = set([start_node])
        closed_list = set([])
        
        # g contains current distances from start_node to all other nodes
        # the default value (if it's not found in the map) is +infinity
        g = {}
        
        g[start_node] = 0
        
        # parents contains an adjancecy map of all nodes
        parents = {}
        parents[start_node] = start_node
        
        while len(open_list) > 0:
            n = None
            
            # find a node with the Lowest value of f() - evaluation function
            for v in open_list:
                if n == None or g[v] + self.h(v) < g[n] + self.h(n):
                    n = v;
                    
            if n == None:
                print('Path does not exist!')
                return None
            
            # if the current node is the stop_node
            # then we begin reconstruction in the path from it to the start_node
            if n == stop_node:
                reconst_path = []
                
                while parents[n] != n:
                    reconst_path.append(n)
                    n = parents[n]
                    
                reconst_path.append(start_node)
                
                reconst_path.reverse()
                
                print('Path found: {}'.format(reconst_path))
                return reconst_path
            
            # for all neighbors of the current node do
            for (m, weight) in self.get_neighbors(n):
                # if the current node isn't in both open_list and closed_list
                # add it to open_list and note n as it's parent
                if m not in open_list and m not in closed_list:
                    open_list.add(m)
                    parents[m] = n
                    g[m] = g[n] + weight
                    
                # otherwise, check if it's quicker to first visit n, then m
                # and if it is, update parent data and g data
                # and if the node was in the closed_list, move it to open_list
                else:
                    if g[m] > g[n] + weight:
                        g[m] = g[n] + weight
                        parents[m] = n
                        
                        if m in closed_list:
                            closed_list.remove(m)
                            open_list.add(m)
                            
            # remove n from the open_list, and add it to closed_list
            # because all of his neighbors were inspected
            open_list.remove(n)
            closed_list.add(n)
            
        print('Path does not exist!')
        return None

adjacency_list = {
    'A' : [('B',1), ('C',3), ('D',7)],
    'B' : [('D',5)],
    'C' : [('D',12)]
}
graph1 = Graph(adjacency_list)
graph1.a_star_algorithm('A','D')

Path found: ['A', 'B', 'D']


['A', 'B', 'D']

## Travelling Salesman Problem

In [2]:
import random
import numpy as np
import math
import time

def makeTSP(nCities):
    positions = 2*np.random.rand(nCities,2)-1;
    distance = np.zeros((nCities,nCities))

    for i in range(nCities):
        for j in range(nCities):
            distance[i,j] = math.sqrt((positions[i,0] - positions[j,0])**2 + (positions[i,1] - positions[j,1])**2);
            distance[j,i] = distance[i,j];
        return distance

def permutation(order):
    order = tuple(order)
    if len(order) == 1:
        yield order
    else:
        for i in range(len(order)):
            rest = order[:i] + order[i+1:]
            move = (order[i],)
            for smaller in permutation(rest):
                yield move + smaller

def exhaustive(distance):
    nCities = np.shape(distance)[0]

    cityOrder = np.arange(nCities)

    distanceTravelled = 0
    for i in range(nCities-1):
        distanceTravelled += distance[cityOrder[i],cityOrder[i+1]]
    distanceTravelled += distance[cityOrder[nCities-1],0]

    for newOrder in permutation(range(nCities)):
        possibleDistanceTravelled = 0
        for i in range(nCities-1):
            possibleDistanceTravelled += distance[newOrder[i],newOrder[i+1]]
        possibleDistanceTravelled += distance[newOrder[nCities-1],0]

        if possibleDistanceTravelled < distanceTravelled:
            distanceTravelled = possibleDistanceTravelled
            cityOrder = newOrder
    return cityOrder, distanceTravelled

def hillClimbing(distance):

    nCities = np.shape(distance)[0]

    cityOrder = np.arange(nCities)
    random.shuffle(cityOrder)

    distanceTravelled = 0
    for i in range(nCities-1):
        distanceTravelled += distance[cityOrder[i],cityOrder[i+1]]
    distanceTravelled += distance[cityOrder[nCities-1],0]

    for i in range(1000):
        # Choose cities to swap
        city1 = np.random.randint(nCities)
        city2 = np.random.randint(nCities)

        if city1 != city2:
            # Reorder the set of cities
            possibleCityOrder = cityOrder.copy()
            possibleCityOrder = np.where(possibleCityOrder==city1,-1,possibleCityOrder)
            possibleCityOrder = np.where(possibleCityOrder==city2,city1,possibleCityOrder)
            possibleCityOrder = np.where(possibleCityOrder==1,city2,possibleCityOrder)

            # Work out the new distance
            # This can be done more efficiently
            newDistanceTravelled = 0
            for j in range(nCities-1):
                newDistanceTravelled += distance[possibleCityOrder[j],possibleCityOrder[j+1]]
            distanceTravelled += distance[cityOrder[nCities-1],0]

            if newDistanceTravelled < distanceTravelled:
                distanceTravelled = newDistanceTravelled
                cityOrder = possibleCityOrder
    return cityOrder, distanceTravelled

def runAll():
    nCities = 5
    distance = makeTSP(nCities)

    print("Exhaustive search")
    start = time.time()
    print(exhaustive(distance))
    finish = time.time()
    print(finish-start)
    print("")
    print("Hill Climbing")
    start = time.time()
    print(hillClimbing(distance))
    finish = time.time()
    print(finish-start)

runAll()

Exhaustive search
((1, 2, 3, 4, 0), 0.8371132877128753)
0.0009984970092773438

Hill Climbing
(array([-1, -1, -1, -1, -1]), 0.0)
0.03293728828430176
